# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

Please make sure if you are running this notebook in the workspace that you have chosen GPU rather than CPU mode.

In [3]:
# Package imports: All the necessary packages and modules are imported in the first cell of the notebook

import time 
import numpy as np

import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import json

from collections import OrderedDict

from PIL import Image

import matplotlib.pyplot as plt
import seaborn as sns

# Configure matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [4]:
# Point to image directories
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [5]:
# Training data augmentation: torchvision transforms are used to augment the training data with random scaling, rotations, mirroring, and/or cropping
# Data normalization: The training, validation, and testing data is appropriately cropped and normalized
# For the means, it's [0.485, 0.456, 0.406] and for the standard deviations [0.229, 0.224, 0.225], calculated from the ImageNet images
# These values will shift each color channel to be centered at 0 and range from -1 to 1

# Training dataset is randomly scaled, transformed, and flipped
train_transforms = transforms.Compose([transforms.RandomRotation(50),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

# Validation and testing datasets share the same transformations with no scaling or rotation
valid_test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])]) 

In [6]:
# Data loading: The data for each set (train, validation, test) is loaded with torchvision's ImageFolder
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
valid_data = datasets.ImageFolder(data_dir + '/valid', transform=valid_test_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=valid_test_transforms)

In [7]:
# Data batching: The data for each set is loaded with torchvision's DataLoader
# Training data is shuffled to prevent unintended bias
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [8]:
# Load in dictionary mapping the integer encoded categories to the actual names of the flowers
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

In [9]:
# Pretrained Network: A pretrained network is loaded from torchvision.models and the parameters are frozen
model = models.vgg16(pretrained = True)
model

# Ensure weights from the pre-trained model are modified by back propagation
for param in model.parameters():
    param.requires_grad = False

# Feedforward Classifier: A new feedforward network is defined for use as a classifier using the features as input
# Define sequential container (skeleton structure) Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout.
# Set first layer input to match the number of features (25088), and output match number of classes i.e. number of flower categories
hidden_layers = 500
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, hidden_layers)),
                          ('relu', nn.ReLU()),
                          ('dropout1', nn.Dropout(0.05)),
                          ('fc2', nn.Linear(hidden_layers, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:04<00:00, 114221267.90it/s]


In [10]:
# Training the network: The parameters of the feedforward classifier are appropriately trained, while the parameters of the feature network are left static

# Begin training on top of pretrained model
model.classifier = classifier

# Set up optimizer: Adam (adaptive moment estimation) to update network weights iteratively based on training data
# Learning rate currently set to default 1e-3 per publication recommendation for use with PyTorch
learning_rate = 1e-3
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)

# Set up loss criterion: infer confidence using negative log likelihood loss because of softmax activation function
criterion = nn.NLLLoss()

# Define validation function for the model
def validation(model, valid_loader, criterion):
    
    accuracy = 0
    valid_loss = 0
    
    # Switch to CUDA
    model.to('cuda')

    # Iterate over images and labels from valid_loader dataset
    for images, labels in valid_loader:
    
        # Switch images and labels to CUDA tensors
        images, labels = images.to('cuda'), labels.to('cuda')

        # Perform forward pass
        output = model.forward(images)
        
        # Calculate validation loss
        valid_loss += criterion(output, labels).item()
        
        # Return a new tensor with the exponential of the elements of the input tensor to calculate probability
        ps = torch.exp(output)
        
        # Calculate accuracy based on the maximum probability versus actual flower label
        match = (labels.data == ps.max(dim=1)[1])
        accuracy += match.type(torch.FloatTensor).mean()
    
    return valid_loss, accuracy

In [11]:
# An epoch = 1 forward pass and 1 backward pass of the full training dataset
epochs = 1

# An iteration = 1 forward and 1 backward pass
iteration = 0
print_line = 40

# Switch to CUDA
model.to('cuda')

for epoch in range(epochs):
    running_loss = 0
    since = time.time()
    
    # Iterate over images and labels from train_loader and valid_loader dataset
    for inputs, labels in train_loader:
        
        iteration += 1
        
        # Convert inputs and labels to cuda tensors
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        
        # Reset gradient back to 0 because here we perform mini-batch gradient descent 
        optimizer.zero_grad()
        
        # Perform forward pass and compute the loss
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        
        # Perform backward pass to compute the gradients with respect to model parameters
        loss.backward()
        
        # Update parameters in the direction of the gradients
        optimizer.step()
        
        # Calculate the training loss
        running_loss += loss.item()
        
        # Validate model
        if iteration % print_line == 0:
            
            # Set model to evaluation mode
            model.eval()
            
            # Turn off gradients to reduce memory usage as it is not needed in validation mode
            with torch.no_grad():
                valid_loss, accuracy = validation(model, valid_loader, criterion)
                
            # Validation Loss and Accuracy: During training, the validation loss and accuracy are displayed
            
            print(f"Epoch {epoch+1}, \
            Training Loss: {round(running_loss/print_line,2)} \
            Validation Loss: {round(valid_loss/len(valid_loader),2)} \
            Validation Accuracy: {round(float(accuracy/len(valid_loader)),2)}")
            
            # Reset training loss
            running_loss = 0
            
            # Set model back to training mode
            model.train()
            
    # Calculate and print time elapsed in each epoch         
    time_elapsed = time.time() - since
    print('Epoch duration: {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

Epoch 1,             Training Loss: 3.27             Validation Loss: 1.38             Validation Accuracy: 0.65
Epoch 1,             Training Loss: 1.67             Validation Loss: 0.85             Validation Accuracy: 0.77
Epoch duration: 3m 13s


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [12]:
def test_model(model):
    # Testing Accuracy: The network's accuracy is measured on the test data        
    accurate = 0
    total = 0

    # Switch to CUDA
    model.to('cuda')

    # Turn off gradients to reduce memory usage as it is not needed in testing mode

    with torch.no_grad():

        for data in test_loader:
            images, labels = data

            # Convert inputs and labels to cuda tensors
            images, labels = images.to('cuda'), labels.to('cuda')

            # Obtain model output values from images
            outputs = model(images)

            # Designate maximum probability output as predicted flower class
            _, predicted = torch.max(outputs.data, 1)

            # Add total number of images to total image counter
            total += labels.size(0)

            # Add accurately classified images to accurate image counter
            accurate += (predicted == labels).sum().item()

        # Print neural network accuracy calculated from accurate/total image counters
        print(f"Neural network accuracy: {round(100 * accurate / total,2)}%")

test_model(model)

Neural network accuracy: 74.97%


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [17]:
# Saving the model: The trained model is saved as a checkpoint along with associated hyperparameters and the class_to_idx dictionary
model.cpu()
checkpoint = {'classifier': model.classifier,
              'state_dict': model.state_dict (),
              'class_to_idx': train_data.class_to_idx
             }

torch.save (checkpoint, 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [19]:
# Loading checkpoints: There is a function that successfully loads a checkpoint and rebuilds the model
checkpoint = torch.load('checkpoint.pth')

model = models.vgg16(pretrained=True)
model.state_dict (checkpoint['state_dict'])
model.classifier = checkpoint['classifier']
model.class_to_idx = checkpoint['class_to_idx']


# Test whether checkpoint loading worked

test_model(model)

Neural network accuracy: 74.73%


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
# Image Processing: The process_image function successfully converts a PIL image into an object that can be used as input to a trained model
def process_image(image):
    
    # Use PIL to load the image 
    pil_image = Image.open(f'{image}' + '.jpg')

    # Normalize image per network expectations: 
    # For the mean, it's [0.485, 0.456, 0.406] and for the standard deviations [0.229, 0.224, 0.225]
    # Resize the images where the shortest side is 256 pixels, keeping the aspect ratio
    # Crop out the center 224x224 portion of the image
    transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], 
                                                         [0.229, 0.224, 0.225])]) 

    pil_transform = transform(pil_image)
    
    # Convert PyTorch tensor to Numpy array 
    np_image = np.array(pil_transform)
    
    return np_image

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

# Test image
image_name = data_dir + '/test' + '/2/' + 'image_05100'
img_test =  process_image(image_name)

# Check to see if image pre-processing was successful
imshow(img_test, ax=None, title=None)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
# Class Prediction: The predict function successfully takes the path to an image and a checkpoint, then returns the top K most probably classes for that image
def predict(image_path, model, k=5):

    # Switch to CPU mode
    loaded_model = load_checkpoint(model).cpu()
    
    # Load and process the image
    img = process_image(image_path)
    
    # Convert from Numpy array to PyTorch tensor
    img_tensor = torch.from_numpy(img).type(torch.FloatTensor)
    
    # Add batch size argument
    model_input = img_tensor.unsqueeze(0)
    
    # Calculate probability using model
    loaded_model.eval()
    with torch.no_grad():
        model_output = loaded_model.forward(model_input)
    
    # Calculate and store top probabilities using torch.topk(k), which returns the highest probabilities and the indices
    probs = torch.exp(model_output)
    probs_topk = probs.topk(k)[0]
    idx_topk = probs.topk(k)[1]
    
    # Convert stored top probabilities to Numpy arrays
    probs_topk_array = np.array(probs_topk)[0]
    idx_topk_array = np.array(idx_topk)[0]
    
    # Convert index to class labels using class_to_idx
    class_to_idx = loaded_model.class_to_idx
    
    # Invert the dictionary to obtain mapping from index to class
    idx_to_class = {val: key for key, val in class_to_idx.items()}
    class_topk_array = []
    for idx in idx_topk_array:
        class_topk_array +=[idx_to_class[idx]]
        
    return probs_topk_array, class_topk_array
        
# Paths to an image and a model checkpoint
image_path = data_dir + '/test' + '/2/' + 'image_05100'
model_path = 'checkpoint.pth' 

# Predict and print the probabilities and classes
probs, classes = predict(image_path, model_path)
print(probs)
print(classes)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# Sanity Checking with matplotlib: A matplotlib figure is created displaying an image and its associated top 5 most probable classes with actual flower names

# Paths to an image and a model checkpoint
image_path = data_dir + '/test' + '/2/' + 'image_05100'
model_path = 'checkpoint.pth' 

# Predict the probabilities and classes
probs, classes = predict(image_path, model_path)

# Convert from the class integer encoding to actual flower names with the cat_to_name.json
names = [cat_to_name[i] for i in classes]

# Load and process the image
img = process_image(image_path)

# Show PyTorch tensor as an image
imshow(img)

# Plot predicted classes using matplotlib and seaborn
plt.figure(figsize = (4,4))
sns.barplot(x=probs, y=names, color= 'darkblue')
plt.show()
